# Яндекс.Музыка

Сравнение Москвы и Петербурга окружено мифами. Например:

`Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
Петербург — культурная столица, со своими вкусами.`

На данных Яндекс.Музыки вы сравните поведение пользователей двух столиц.

Цель исследования — проверьте три гипотезы:

`Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.
Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.`

Ход исследования

Данные о поведении пользователей вы получите из файла yandex_music_project.csv. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных.

Вы проверите данные на ошибки и оцените их влияние на исследование. Затем, на этапе предобработки вы поищете возможность исправить самые критичные ошибки данных.

Таким образом, исследование пройдёт в три этапа:
    
- Обзор данных.
- Предобработка данных.
- Проверка гипотез.

## 1. Обзор данных

In [51]:
import pandas as pd

df = pd.read_csv('yandex_music_project.csv')

df.head(10)


,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — object.

Согласно документации к данным:

`userID — идентификатор пользователя;
Track — название трека;
artist — имя исполнителя;
genre — название жанра;
City — город пользователя;
time — время начала прослушивания;
Day — день недели.`


В названиях колонок видны три нарушения стиля:

`Строчные буквы сочетаются с прописными.
Встречаются пробелы.
Найдите ещё одну проблему в названии колонок и опишите её в этом пункте.`

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

Выводы

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## 2 Предобработка данных

Исправьте стиль в заголовках столбцов, исключите пропуски. Затем проверьте данные на дубликаты.

### 2.1 Стиль заголовков

Выведите на экран названия столбцов:

In [53]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [54]:
df = df.rename(columns = {'  userID':'user_id', 'Track':'track', '  City  ':'city', 'Day':'day'})
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### 2.2 Пропуски значений

Сначала посчитайте, сколько в таблице пропущенных значений. Для этого достаточно двух методов pandas:

In [55]:
df.isnull().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Не все пропущенные значения влияют на исследование. Так в track и artist пропуски не важны для вашей работы. Достаточно заменить их явными обозначениями.

Но пропуски в genre могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:

`заполнить и эти пропуски явными обозначениями, оценить, насколько они повредят расчётам.`


In [56]:
columns_to_replace = ['track','artist','genre']

for col in columns_to_replace:
    df[col] = df[col].fillna('unknown')
df.isnull().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

Подсчёт явных дубликатов

In [57]:
df.duplicated().sum()

3826

In [58]:
df = df.drop_duplicates().reset_index(drop=True)

In [59]:
df.duplicated().sum()

0

Теперь избавьтесь от неявных дубликатов в колонке genre. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Выведите на экран список уникальных названий жанров, отсортированный в алфавитном порядке. Для этого:

`извлеките нужный столбец датафрейма,
примените к нему метод сортировки,
для отсортированного столбца вызовите метод, который вернёт уникальные значения из столбца.`


In [60]:
genre_name = df['genre'].unique()
genre_name

array(['rock', 'pop', 'folk', 'dance', 'rusrap', 'ruspop', 'world',
       'electronic', 'unknown', 'alternative', 'children', 'rnb', 'hip',
       'jazz', 'postrock', 'latin', 'classical', 'metal', 'reggae',
       'tatar', 'blues', 'instrumental', 'rusrock', 'dnb', 'türk', 'post',
       'country', 'psychedelic', 'conjazz', 'indie', 'posthardcore',
       'local', 'avantgarde', 'punk', 'videogame', 'techno', 'house',
       'christmas', 'melodic', 'caucasian', 'reggaeton', 'soundtrack',
       'singer', 'ska', 'shanson', 'ambient', 'film', 'western', 'rap',
       'beats', "hard'n'heavy", 'progmetal', 'minimal', 'contemporary',
       'new', 'soul', 'holiday', 'german', 'tropical', 'fairytail',
       'spiritual', 'urban', 'gospel', 'nujazz', 'folkmetal', 'trance',
       'miscellaneous', 'anime', 'hardcore', 'progressive', 'chanson',
       'numetal', 'vocal', 'estrada', 'russian', 'classicmetal',
       'dubstep', 'club', 'deep', 'southern', 'black', 'folkrock',
       'fitness', '

Просмотрите список и найдите неявные дубликаты названия hiphop. Это могут быть названия с ошибками или альтернативные названия того же жанра.

Вы увидите следующие неявные дубликаты:

`hip,
hop,
hip-hop.`

Чтобы очистить от них таблицу, напишите функцию replace_wrong_genres() с двумя параметрами:

`wrong_genres — список дубликатов,
correct_genre — строка с правильным значением.`


In [61]:
def replace_wrong_genres(wrong_genres, correct_genres):
    for i in wrong_genres:
        df['genre'] = df['genre'].replace(i,correct_genres)

In [62]:
replace_wrong_genres(['hip','hop','hip-hop'],'hiphop')

In [63]:
genre_name = df['genre'].unique()
genre_name

array(['rock', 'pop', 'folk', 'dance', 'rusrap', 'ruspop', 'world',
       'electronic', 'unknown', 'alternative', 'children', 'rnb',
       'hiphop', 'jazz', 'postrock', 'latin', 'classical', 'metal',
       'reggae', 'tatar', 'blues', 'instrumental', 'rusrock', 'dnb',
       'türk', 'post', 'country', 'psychedelic', 'conjazz', 'indie',
       'posthardcore', 'local', 'avantgarde', 'punk', 'videogame',
       'techno', 'house', 'christmas', 'melodic', 'caucasian',
       'reggaeton', 'soundtrack', 'singer', 'ska', 'shanson', 'ambient',
       'film', 'western', 'rap', 'beats', "hard'n'heavy", 'progmetal',
       'minimal', 'contemporary', 'new', 'soul', 'holiday', 'german',
       'tropical', 'fairytail', 'spiritual', 'urban', 'gospel', 'nujazz',
       'folkmetal', 'trance', 'miscellaneous', 'anime', 'hardcore',
       'progressive', 'chanson', 'numetal', 'vocal', 'estrada', 'russian',
       'classicmetal', 'dubstep', 'club', 'deep', 'southern', 'black',
       'folkrock', 'fitness'

Выводы

Предобработка обнаружила три проблемы в данных:

`нарушения в стиле заголовков,
пропущенные значения,
дубликаты — явные и неявные.`

Вы исправили заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения вы заменили на 'unknown'. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке genre.

Теперь можно перейти к проверке гипотез.

## 3 Проверка гипотез
### 3.1 Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверьте это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

`Разделите пользователей Москвы и Санкт-Петербурга
Сравните, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.`

Для тренировки сначала выполните каждый из расчётов по отдельности.

Оцените активность пользователей в каждом городе. Сгруппируйте данные по городу и посчитайте прослушивания в каждой группе.

In [64]:
df.groupby('city')['genre'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: genre, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируйте данные по дню недели и подсчитайте прослушивания в понедельник, среду и пятницу. Учтите, что в данных есть информация о прослушиваниях только за эти дни.

In [65]:
df.groupby('day')['genre'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: genre, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Вы видели, как работает группировка по городу и по дням недели. Теперь напишите функцию, которая объединит два эти расчёта.

Создайте функцию number_tracks(), которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:

- день недели,
- название города.

В функции сохраните в переменную строки исходной таблицы, у которых значение:

- в колонке day равно параметру day,
- в колонке city равно параметру city.

Для этого примените последовательную фильтрацию с логической индексацией.

Затем посчитайте значения в столбце user_id получившейся таблицы. Результат сохраните в новую переменную. Верните эту переменную из функции.

Объявляется функция с двумя параметрами: day, city.
В переменной track_list сохраняются те строки таблицы df, для которых
значение в столбце 'day' равно параметру day и одновременно значение
в столбце 'city' равно параметру city (используйте последовательную фильтрацию
с помощью логической индексации).
В переменной track_list_count сохраняется число значений столбца 'user_id',
рассчитанное методом count() для таблицы track_list.
Функция возвращает число - значение track_list_count.


Функция для подсчёта прослушиваний для конкретного города и дня.
С помощью последовательной фильтрации с логической индексацией она
сначала получит из исходной таблицы строки с нужным днём,
затем из результата отфильтрует строки с нужным городом,
методом count() посчитает количество значений в колонке user_id.
Это количество функция вернёт в качестве результата

In [66]:
def number_tracks(day, city):
    track_list = df[(df['day'] == day) & (df['city'] == city)]
    track_list_count = track_list['genre'].count()
    return track_list_count

In [67]:
number_tracks('Monday','Moscow')

15740

In [68]:
number_tracks('Monday','Saint-Petersburg')

5614

In [69]:
number_tracks('Wednesday','Moscow')

11056

In [35]:
number_tracks('Wednesday','Saint-Petersburg')

7003

In [70]:
number_tracks('Friday','Moscow')

15945

In [37]:
number_tracks('Friday','Saint-Petersburg')

5895

Создайте c помощью конструктора pd.DataFrame таблицу, где

`названия колонок — ['city', 'monday', 'wednesday', 'friday'];
данные — результаты, которые вы получили с помощью number_tracks.`


In [71]:
columns = ['city', 'monday', 'wednesday', 'friday']

data = [['Moscow', 15740, 11056, 15945],
        ['Saint-Petersburg', 5614, 7003, 5895]]

In [72]:
table = pd.DataFrame(data = data, columns = columns)
table

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


Выводы

Данные показывают разницу поведения пользователей:

`В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.`

Значит, данные говорят в пользу первой гипотезы.

## 3.2 Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраните таблицы с данными в две переменные:

- по Москве — в moscow_general;
- по Санкт-Петербургу — в spb_general.


In [73]:
moscow_general = df[df['city'] == 'Moscow']

In [74]:
spb_general = df[df['city'] == 'Saint-Petersburg']

Создайте функцию genre_weekday() с четырьмя параметрами:

- таблица (датафрейм) с данными,
- день недели,
- начальная временная метка в формате 'hh:mm',
- последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [76]:
def genre_weekday(table, day, time1, time2):
    genre_df = table[(table['day'] == day) & (table['time'] > time1) & (table['time'] < time2)]
    genre_df_count = genre_df.groupby('genre')['user_id'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending = False)
    return genre_df_sorted.head(10)


In [77]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: user_id, dtype: int64

In [78]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: user_id, dtype: int64

In [79]:
genre_weekday(moscow_general, 'Friday', '18:00', '22:00')

genre
pop            710
rock           508
dance          489
electronic     478
hiphop         273
world          206
ruspop         169
alternative    163
classical      160
rusrap         139
Name: user_id, dtype: int64

In [80]:
genre_weekday(spb_general, 'Monday', '18:00', '22:00')

genre
pop            259
rock           206
electronic     190
dance          188
hiphop         104
alternative     71
classical       70
jazz            57
rusrap          54
ruspop          53
Name: user_id, dtype: int64

Выводы

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

- В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

- В Москве пропущенных значений оказалось так много, что значение 'unknown' заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:

- Пользователи слушают похожую музыку в начале недели и в конце.
- Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные данные о жанрах.

## 3.3 Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве. А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируйте таблицу moscow_general по жанру и посчитайте прослушивания треков каждого жанра методом count(). Затем отсортируйте результат в порядке убывания и сохраните его в таблице moscow_genres.

`одной строкой: группировка таблицы moscow_general по столбцу 'genre',
подсчёт числа значений 'genre' в этой группировке методом count(),
сортировка получившегося Series в порядке убывания и сохранение в moscow_genres`

In [81]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending = False)

In [82]:
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

In [83]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending = False)

In [84]:
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

Выводы

Гипотеза частично подтвердилась:

- Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
- Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.


# 4 Итоги исследования

Вы проверили три гипотезы и установили:

- День недели по-разному влияет на активность пользователей в Москве и Петербурге.

Первая гипотеза полностью подтвердилась.

- Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:

- в Москве слушают музыку жанра “world”,
- в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

- Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

На практике исследования содержат проверки статистических гипотез. Из данных одного сервиса не всегда можно сделать вывод о всех жителях города. Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных. С методами проверок гипотез вы ещё познакомитесь в следующих темах.